In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os

import torch
import torchvision
from torchvision import datasets, transforms
import torch.utils
import torch.utils.data 
import torch.utils.data.dataloader
import torchvision.models.resnet

from sklearn.metrics import roc_curve, auc, accuracy_score
from sklearn.metrics import f1_score
from itertools import groupby

In [2]:
print(f"PyTorch version: {torch.__version__}")

PyTorch version: 2.4.0


In [3]:
print(f"Is MPS built? {torch.backends.mps.is_built()}")
print(f"Is MPS available? {torch.backends.mps.is_available()}")

# Set device
device = "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using device: {device}")

Is MPS built? True
Is MPS available? True
Using device: mps


In [4]:
def validate(model, loader):
    correct = 0
    total = 0
    
    model.eval()
    with torch.no_grad():
        for images, labels in loader:
            # Send data to device
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
    return correct / total

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split

# 2. Laden des model Modells ohne vortrainierte Gewichte
model = models.alexnet(pretrained=True)
#model = models.resnet50(pretrained=True)
#model = models.vgg16(pretrained=True)
model = model.to(device)

# 3. & 4. Datensatz vorbereiten und in Trainings- und Validierungssets aufteilen
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# 5. Laden des Datensatzes

# Absoluter Pfad basierend auf dem aktuellen Arbeitsverzeichnis
parent_directory = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Relative path to the folder containing the images
relative_folderpath = os.path.join(parent_directory, 'data/segmented/graph/pl')

# Erstellen des Datasets
dataset = datasets.ImageFolder(relative_folderpath, transform=transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True, num_workers=4)

random_seed = 2024
torch.manual_seed(random_seed)

# Define the sizes of the train and test sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

# Split the dataset into train and test sets
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
print('Train Dataset: ' + str(len(train_dataset)))
print('Test Dataset: ' + str(len(test_dataset)))


train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=True, num_workers=4)

# 6. Verlustfunktion und Optimierer definieren
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# 7. Trainingsschleife (vereinfacht)
for epoch in range(10):
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
    train_accuracy = validate(model, test_loader)
    print(f'Epoch: {epoch + 1} | Loss: {loss.item()} | Train Accuracy:  {train_accuracy}')


/Users/davidpichler/GitHubRepo/Thesis/.conda/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/davidpichler/GitHubRepo/Thesis/.conda/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Train Dataset: 719
Test Dataset: 180
Epoch: 1 | Loss: 1.445928692817688 | Train Accuracy:  0.3333333333333333
Epoch: 2 | Loss: 1.1362907886505127 | Train Accuracy:  0.3333333333333333
Epoch: 3 | Loss: 1.1817418336868286 | Train Accuracy:  0.34444444444444444
Epoch: 4 | Loss: 1.3222887516021729 | Train Accuracy:  0.3333333333333333
Epoch: 5 | Loss: 1.2287122011184692 | Train Accuracy:  0.32222222222222224
Epoch: 6 | Loss: 1.2355290651321411 | Train Accuracy:  0.34444444444444444
Epoch: 7 | Loss: 1.263938069343567 | Train Accuracy:  0.34444444444444444
Epoch: 8 | Loss: 1.0792806148529053 | Train Accuracy:  0.32222222222222224
Epoch: 9 | Loss: 1.168463110923767 | Train Accuracy:  0.34444444444444444
Epoch: 10 | Loss: 1.4112025499343872 | Train Accuracy:  0.37222222222222223


In [9]:
model.eval()

with torch.no_grad():
   correct = 0
   total = 0
   true_labels = []
   predicted_labels = []

   for X_test_tensor, Y_test_tensor in test_loader:
      X_test_tensor, Y_test_tensor = X_test_tensor.to(device), Y_test_tensor.to(device)
      outputs = model(X_test_tensor)
      _, predicted = torch.max(outputs.data, 1)

      y_test = Y_test_tensor.cpu().numpy()
      predicted = predicted.cpu().numpy()

      correct += (predicted == y_test).sum().item()
      total += len(y_test)


      true_labels.extend(y_test)
      predicted_labels.extend(predicted)

   unique, counts = np.unique(predicted_labels, return_counts=True)

   print(f'Predicted labels: {dict(zip(unique, counts))}')

   unique, counts = np.unique(true_labels, return_counts=True)
   print(f'True labels: {dict(zip(unique, counts))}')

   accuracy = correct / total

   print(f'Accuracy: {accuracy}')

   f1 = f1_score(true_labels, predicted_labels, average='weighted')
   print(f'F1 Score: {f1}')


Predicted labels: {np.int64(1): np.int64(78), np.int64(2): np.int64(102)}
True labels: {np.int64(0): np.int64(60), np.int64(1): np.int64(58), np.int64(2): np.int64(62)}
Accuracy: 0.37222222222222223
F1 Score: 0.29596285668739036


In [8]:
import pandas as pd
from sklearn.metrics import confusion_matrix

# Create a DataFrame from the confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)
cm_df = pd.DataFrame(cm, index=['Actual Corrected', 'Actual Original', 'Actual Smoothed'], columns=['Predicted Corrected', 'Predicted Original', 'Predicted Smoothed'])

# Display the DataFrame
print(cm_df)


                  Predicted Corrected  Predicted Original  Predicted Smoothed
Actual Corrected                    0                  29                  31
Actual Original                     0                  27                  31
Actual Smoothed                     0                  22                  40


In [ ]:
if not os.path.exists('model/'): os.makedirs('model/')
torch.save({
    'model_state_dict': model.state_dict(),
    'model_architecture': model
}, 'model/AlexNet_complete.pth')

In [ ]:
'''
 if average_validation_loss < best_validation_loss:
            best_validation_loss = average_validation_loss
            best_model_state_dict = model.state_dict()
            print(epoch, ' saved!')
'''

"\n if average_validation_loss < best_validation_loss:\n            best_validation_loss = average_validation_loss\n            best_model_state_dict = model.state_dict()\n            print(epoch, ' saved!')\n"